## Federated Learning with one corrupted node 
Using one balanced and one unbalanced dataset with one corrupted node (5%, 25% and 50% corrupted samples) to test different aggregation functions and determine the more robust one. 

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 2), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

### AVERAGE 3 nodes

In [5]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [6]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')
test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3457440085.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3457440085.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_complete = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Complete.csv')


In [7]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)
xcomplete, ycomplete = preprocessing(test_complete)

#### 3C 5%

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%13C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%13C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%13C-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_956/3517758634.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%13C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_956/3517758634.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%13C-Part3.csv')


In [9]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr53Cavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr53Cavg.hdf5')

Epoch 1/5
51/51 [==============================] - 9s 168ms/step - loss: 0.3733 - accuracy: 0.8686 - val_loss: 0.1828 - val_accuracy: 0.9563
Epoch 2/5
51/51 [==============================] - 8s 160ms/step - loss: 0.1649 - accuracy: 0.9643 - val_loss: 0.1570 - val_accuracy: 0.9658
Epoch 3/5
51/51 [==============================] - 8s 165ms/step - loss: 0.1491 - accuracy: 0.9671 - val_loss: 0.1489 - val_accuracy: 0.9665
Epoch 4/5
51/51 [==============================] - 8s 164ms/step - loss: 0.1431 - accuracy: 0.9675 - val_loss: 0.1453 - val_accuracy: 0.9670
Epoch 5/5
51/51 [==============================] - 8s 163ms/step - loss: 0.1407 - accuracy: 0.9679 - val_loss: 0.1451 - val_accuracy: 0.9670
Epoch 1/5
51/51 [==============================] - 9s 174ms/step - loss: 0.0224 - accuracy: 0.9921 - val_loss: 0.2045 - val_accuracy: 0.9179
Epoch 2/5
51/51 [==============================] - 9s 175ms/step - loss: 0.0185 - accuracy: 0.9935 - val_loss: 0.2520 - val_accuracy: 0.8357
Epoch 3/5
51/

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.13129368424415588, 0.29392746090888977], [0.21367402374744415, 0.4640328288078308], [0.25993427634239197, 0.6341381669044495]]
Accuracy for iterations:  [[0.9815361499786377, 0.9085228443145752], [0.9505828619003296, 0.8299789428710938], [0.918840765953064, 0.7634418606758118]]
F1 for iterations:  [[0.981547115873357, 0.90779772048695], [0.9504388438741762, 0.823371275255111], [0.9180986447905306, 0.7449847862228615]]
Precision for iterations:  [[0.9819145017487854, 0.9110732237836993], [0.9522931983987097, 0.8535038895297559], [0.9267800594450063, 0.8119110396538817]]
Recall for iterations:  [[0.9815361243390108, 0.908522851122575], [0.9505828391130328, 0.82997892901257], [0.918840749072424, 0.7634418368088353]]


#### 3C 25%

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%13C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%13C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%13C-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/1190975088.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%13C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/1190975088.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%13C-Part3.csv')


In [17]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr253Cavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr253Cavg.hdf5')

Epoch 1/5
51/51 [==============================] - 9s 181ms/step - loss: 0.5087 - accuracy: 0.7875 - val_loss: 0.4147 - val_accuracy: 0.8653
Epoch 2/5
51/51 [==============================] - 11s 206ms/step - loss: 0.4059 - accuracy: 0.8672 - val_loss: 0.3999 - val_accuracy: 0.8674
Epoch 3/5
51/51 [==============================] - 11s 212ms/step - loss: 0.3952 - accuracy: 0.8682 - val_loss: 0.3925 - val_accuracy: 0.8682
Epoch 4/5
51/51 [==============================] - 12s 227ms/step - loss: 0.3912 - accuracy: 0.8685 - val_loss: 0.3905 - val_accuracy: 0.8684
Epoch 5/5
51/51 [==============================] - 11s 209ms/step - loss: 0.3898 - accuracy: 0.8687 - val_loss: 0.3904 - val_accuracy: 0.8686
Epoch 1/5
51/51 [==============================] - 10s 202ms/step - loss: 0.0336 - accuracy: 0.9910 - val_loss: 0.2968 - val_accuracy: 0.8210
Epoch 2/5
51/51 [==============================] - 10s 203ms/step - loss: 0.0212 - accuracy: 0.9919 - val_loss: 0.3161 - val_accuracy: 0.7573
Epoch 3

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06029896065592766, 0.23395667970180511], [0.06586694717407227, 0.23241287469863892], [0.0801391676068306, 0.45889586210250854]]
Accuracy for iterations:  [[0.9876128435134888, 0.9194579720497131], [0.9834350943565369, 0.911392867565155], [0.9839609861373901, 0.8010426759719849]]
F1 for iterations:  [[0.9876179873106058, 0.9186358495776776], [0.9834448854405644, 0.9104962770279796], [0.9839582869405543, 0.7884540368927155]]
Precision for iterations:  [[0.9877636771386369, 0.923821019409445], [0.9838040603809045, 0.9154707842555262], [0.983970662963597, 0.844334649351584]]
Recall for iterations:  [[0.987612842910982, 0.9194579670130059], [0.9834350988927517, 0.9113928649277047], [0.9839609687691724, 0.8010426505849015]]


#### 3C 50%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%13C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%13C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%13C-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/4238720402.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%13C-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/4238720402.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%13C-Part3.csv')


In [25]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr503Cavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr503Cavg.hdf5')

Epoch 1/5
51/51 [==============================] - 8s 163ms/step - loss: 0.6068 - accuracy: 0.7018 - val_loss: 0.5729 - val_accuracy: 0.7462
Epoch 2/5
51/51 [==============================] - 8s 151ms/step - loss: 0.5733 - accuracy: 0.7447 - val_loss: 0.5688 - val_accuracy: 0.7473
Epoch 3/5
51/51 [==============================] - 8s 159ms/step - loss: 0.5701 - accuracy: 0.7450 - val_loss: 0.5663 - val_accuracy: 0.7476
Epoch 4/5
51/51 [==============================] - 8s 163ms/step - loss: 0.5690 - accuracy: 0.7454 - val_loss: 0.5663 - val_accuracy: 0.7475
Epoch 5/5
51/51 [==============================] - 9s 182ms/step - loss: 0.5690 - accuracy: 0.7455 - val_loss: 0.5664 - val_accuracy: 0.7476
Epoch 1/5
51/51 [==============================] - 9s 169ms/step - loss: 0.0554 - accuracy: 0.9908 - val_loss: 0.4252 - val_accuracy: 0.6590
Epoch 2/5
51/51 [==============================] - 9s 184ms/step - loss: 0.0218 - accuracy: 0.9917 - val_loss: 0.3304 - val_accuracy: 0.7486
Epoch 3/5
51/

In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07479007542133331, 0.2228516787290573], [0.10720720887184143, 0.3528783619403839], [0.1736885905265808, 0.47727009654045105]]
Accuracy for iterations:  [[0.985136866569519, 0.9198939204216003], [0.9881094694137573, 0.8688876032829285], [0.9779719114303589, 0.7984269261360168]]
F1 for iterations:  [[0.9851443052890743, 0.919223630582906], [0.9881124299420956, 0.8653072885389217], [0.9779549154317135, 0.7853341121432225]]
Precision for iterations:  [[0.9853745264525866, 0.9229680470935299], [0.9881581013337662, 0.8855111079149852], [0.9781830540609326, 0.8429088245423307]]
Recall for iterations:  [[0.985136872242835, 0.9198939184770762], [0.988109497794268, 0.8688875971808472], [0.9779718951766047, 0.7984269418004796]]


#### 3G 5%

In [32]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%23G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%23G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%23G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/2562238320.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%23G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/2562238320.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%23G-Part3.csv')


In [33]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [34]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr53Gavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [35]:
global_model = build_model((24,1,1))

In [36]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

In [37]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [38]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr53Gavg.hdf5')

Epoch 1/5
20/20 [==============================] - 5s 247ms/step - loss: 0.5260 - accuracy: 0.9330 - val_loss: 1.5660 - val_accuracy: 0.1458
Epoch 2/5
20/20 [==============================] - 5s 229ms/step - loss: 0.1905 - accuracy: 0.9545 - val_loss: 1.2389 - val_accuracy: 0.1735
Epoch 3/5
20/20 [==============================] - 5s 231ms/step - loss: 0.0837 - accuracy: 0.9754 - val_loss: 0.6450 - val_accuracy: 0.7148
Epoch 4/5
20/20 [==============================] - 5s 250ms/step - loss: 0.0592 - accuracy: 0.9796 - val_loss: 0.5175 - val_accuracy: 0.7650
Epoch 5/5
20/20 [==============================] - 5s 236ms/step - loss: 0.0487 - accuracy: 0.9832 - val_loss: 0.4251 - val_accuracy: 0.8387
Epoch 1/5
54/54 [==============================] - 13s 244ms/step - loss: 0.1586 - accuracy: 0.9655 - val_loss: 0.1453 - val_accuracy: 0.9679
Epoch 2/5
54/54 [==============================] - 13s 243ms/step - loss: 0.1446 - accuracy: 0.9673 - val_loss: 0.1391 - val_accuracy: 0.9686
Epoch 3/5
5

In [39]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.3104642331600189, 0.4340478181838989], [0.32722339034080505, 0.3996964395046234], [0.1822798252105713, 0.41584157943725586]]
Accuracy for iterations:  [[0.8650486469268799, 0.8596236109733582], [0.8397119641304016, 0.8562995195388794], [0.9735750555992126, 0.8286710977554321]]
F1 for iterations:  [[0.8625695297943645, 0.856015643855784], [0.8350657276233903, 0.8515486511560322], [0.9735519267354799, 0.8205135437845434]]
Precision for iterations:  [[0.8816240653743856, 0.8739812836904703], [0.8671038802962394, 0.8773189481505456], [0.9738316774014399, 0.8608133572667405]]
Recall for iterations:  [[0.8650486429635689, 0.8596236285693526], [0.839711940167694, 0.8562994986558163], [0.973575038709866, 0.8286710746203589]]


#### 3G 25%

In [40]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%23G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%23G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%23G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/3341559854.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%23G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/3341559854.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%23G-Part3.csv')


In [41]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [42]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr253Gavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [43]:
global_model = build_model((24,1,1))

In [44]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

In [45]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [46]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr253Gavg.hdf5')

Epoch 1/5
20/20 [==============================] - 5s 245ms/step - loss: 0.5533 - accuracy: 0.9321 - val_loss: 1.4477 - val_accuracy: 0.1457
Epoch 2/5
20/20 [==============================] - 5s 232ms/step - loss: 0.2214 - accuracy: 0.9325 - val_loss: 1.7663 - val_accuracy: 0.1462
Epoch 3/5
20/20 [==============================] - 5s 233ms/step - loss: 0.0931 - accuracy: 0.9742 - val_loss: 0.5407 - val_accuracy: 0.7873
Epoch 4/5
20/20 [==============================] - 5s 235ms/step - loss: 0.0585 - accuracy: 0.9816 - val_loss: 0.5089 - val_accuracy: 0.7737
Epoch 5/5
20/20 [==============================] - 5s 242ms/step - loss: 0.0476 - accuracy: 0.9834 - val_loss: 0.3371 - val_accuracy: 0.9068
Epoch 1/5
54/54 [==============================] - 14s 262ms/step - loss: 0.4473 - accuracy: 0.8628 - val_loss: 0.4032 - val_accuracy: 0.8685
Epoch 2/5
54/54 [==============================] - 15s 270ms/step - loss: 0.3979 - accuracy: 0.8686 - val_loss: 0.3950 - val_accuracy: 0.8682
Epoch 3/5
5

In [47]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.060935135930776596, 0.15702813863754272], [0.09976464509963989, 0.2867684066295624], [0.15649591386318207, 0.3756396770477295]]
Accuracy for iterations:  [[0.9877881407737732, 0.9371866583824158], [0.9872987866401672, 0.90207439661026], [0.9864296317100525, 0.8681973218917847]]
F1 for iterations:  [[0.9877931826222887, 0.9367851562505448], [0.9873033816435282, 0.9005431200450676], [0.9864313300484122, 0.8643057549560927]]
Precision for iterations:  [[0.9879373458999762, 0.9393157774336919], [0.9874095350750886, 0.9101737793375498], [0.986443035436232, 0.886921073906556]]
Recall for iterations:  [[0.9877881328697887, 0.9371866598851994], [0.9872987817347862, 0.9020744023832014], [0.9864296356890356, 0.8681973406960691]]


#### 3G 50%

In [48]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%23G-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%23G-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%23G-Part3.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/951321220.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%23G-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3796/951321220.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr50%23G-Part3.csv')


In [49]:
# Define 
node_datasets = [training1, training2, training3]
num_nodes = 3
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [50]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr503Gavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [51]:
global_model = build_model((24,1,1))

In [52]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)

In [53]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [54]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        else:
            x, y = x3, y3
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr503Gavg.hdf5')

Epoch 1/5
20/20 [==============================] - 5s 250ms/step - loss: 0.5215 - accuracy: 0.9168 - val_loss: 1.6026 - val_accuracy: 0.1469
Epoch 2/5
20/20 [==============================] - 5s 236ms/step - loss: 0.1918 - accuracy: 0.9485 - val_loss: 1.2337 - val_accuracy: 0.1804
Epoch 3/5
20/20 [==============================] - 5s 233ms/step - loss: 0.0811 - accuracy: 0.9772 - val_loss: 0.5493 - val_accuracy: 0.7746
Epoch 4/5
20/20 [==============================] - 5s 234ms/step - loss: 0.0549 - accuracy: 0.9805 - val_loss: 0.4581 - val_accuracy: 0.8127
Epoch 5/5
20/20 [==============================] - 5s 240ms/step - loss: 0.0465 - accuracy: 0.9844 - val_loss: 0.4765 - val_accuracy: 0.8006
Epoch 1/5
54/54 [==============================] - 14s 253ms/step - loss: 0.7051 - accuracy: 0.7324 - val_loss: 0.5756 - val_accuracy: 0.7452
Epoch 2/5
54/54 [==============================] - 15s 274ms/step - loss: 0.5740 - accuracy: 0.7457 - val_loss: 0.5715 - val_accuracy: 0.7458
Epoch 3/5
5

In [55]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.05575968325138092, 0.11677917838096619], [0.06265713274478912, 0.20614604651927948], [0.09469029307365417, 0.34547773003578186]]
Accuracy for iterations:  [[0.9883213043212891, 0.9602376222610474], [0.9891027808189392, 0.9223642945289612], [0.9889713525772095, 0.8676705956459045]]
F1 for iterations:  [[0.9883261113387183, 0.9601533630651375], [0.9891066536238714, 0.9215314952400294], [0.9889740519129036, 0.8636963260637445]]
Precision for iterations:  [[0.9884684140142277, 0.9605227937723785], [0.9892063168439134, 0.9272031121527623], [0.9890192719638692, 0.886847151134282]]
Recall for iterations:  [[0.988321306494493, 0.9602375935479184], [0.9891028075608402, 0.9223643101068081], [0.988971340091735, 0.8676705660103176]]


## CORRUPTION WITH 5 NODES 
Trying aggregation functions with corrupted partitions

### FEDAVG 

In [8]:
def aggregate(grad_list):
    # Calculate mean gradient for each layer
    mean_grad = [tf.reduce_mean(layer_grads, axis=0) for layer_grads in zip(*grad_list)]
    
    return mean_grad

In [14]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/3836997398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [15]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

#### 5A 5% 

In [9]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3687049449.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3687049449.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3687049449.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3687049449.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on im

In [10]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [11]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Aavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [12]:
global_model = build_model((24,1,1))

In [13]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [14]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [15]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Aavg.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 190ms/step - loss: 0.4766 - accuracy: 0.8151 - val_loss: 0.2719 - val_accuracy: 0.8733
Epoch 2/5
31/31 [==============================] - 5s 176ms/step - loss: 0.1992 - accuracy: 0.9477 - val_loss: 0.1789 - val_accuracy: 0.9603
Epoch 3/5
31/31 [==============================] - 6s 202ms/step - loss: 0.1656 - accuracy: 0.9651 - val_loss: 0.1630 - val_accuracy: 0.9632
Epoch 4/5
31/31 [==============================] - 6s 193ms/step - loss: 0.1541 - accuracy: 0.9661 - val_loss: 0.1539 - val_accuracy: 0.9642
Epoch 5/5
31/31 [==============================] - 6s 197ms/step - loss: 0.1469 - accuracy: 0.9674 - val_loss: 0.1490 - val_accuracy: 0.9655
Epoch 1/5
31/31 [==============================] - 7s 218ms/step - loss: 0.0334 - accuracy: 0.9900 - val_loss: 0.5704 - val_accuracy: 0.6368
Epoch 2/5
31/31 [==============================] - 6s 202ms/step - loss: 0.0235 - accuracy: 0.9912 - val_loss: 0.3290 - val_accuracy: 0.7542
Epoch 3/5
31/

In [16]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.057268597185611725, 0.3753368854522705, 0.10097719728946686], [0.05764966458082199, 0.41941025853157043, 0.10066546499729156], [0.06108490750193596, 0.3515525162220001, 0.1044953316450119]]
Accuracy for iterations:  [[0.9712086319923401, 0.8188984990119934, 0.9490266442298889], [0.973151445388794, 0.7767383456230164, 0.952339768409729], [0.9738671779632568, 0.8170820474624634, 0.9537696242332458]]
F1 for iterations:  [[0.9711589716015901, 0.8093341715665523, 0.9489296911792579], [0.9731106298506226, 0.7590747621452839, 0.9522625656682077], [0.9738287441158313, 0.8071441568178996, 0.9537007975221027]]
Precision for iterations:  [[0.97202735364702, 0.8547832333134442, 0.9524616779800035], [0.9738223143211889, 0.8306946500689956, 0.9552843739652934], [0.9745054735122656, 0.8542347118500215, 0.9564842558409361]]
Recall for iterations:  [[0.9712086242659733, 0.8188984959674489, 0.9490266375576311], [0.973151421309416, 0.7767383564629805, 0.9523397666162141], [0.973

#### 5B 10%

In [17]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/2463029928.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/2463029928.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/2463029928.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/2463029928.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [18]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [19]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Aavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [20]:
global_model = build_model((24,1,1))

In [21]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [22]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [23]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Aavg.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 197ms/step - loss: 0.4756 - accuracy: 0.8252 - val_loss: 0.2809 - val_accuracy: 0.9237
Epoch 2/5
31/31 [==============================] - 6s 198ms/step - loss: 0.2623 - accuracy: 0.9305 - val_loss: 0.2468 - val_accuracy: 0.9396
Epoch 3/5
31/31 [==============================] - 5s 171ms/step - loss: 0.2411 - accuracy: 0.9403 - val_loss: 0.2325 - val_accuracy: 0.9421
Epoch 4/5
31/31 [==============================] - 6s 204ms/step - loss: 0.2299 - accuracy: 0.9423 - val_loss: 0.2248 - val_accuracy: 0.9426
Epoch 5/5
31/31 [==============================] - 5s 173ms/step - loss: 0.2237 - accuracy: 0.9427 - val_loss: 0.2218 - val_accuracy: 0.9426
Epoch 1/5
31/31 [==============================] - 7s 216ms/step - loss: 0.0401 - accuracy: 0.9892 - val_loss: 0.4594 - val_accuracy: 0.7053
Epoch 2/5
31/31 [==============================] - 6s 187ms/step - loss: 0.0238 - accuracy: 0.9905 - val_loss: 0.4849 - val_accuracy: 0.6429
Epoch 3/5
31/

In [24]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.04630370810627937, 0.2961485683917999, 0.08095862716436386], [0.05615843087434769, 0.354496031999588, 0.0996839851140976], [0.06967361271381378, 0.34686166048049927, 0.11991321295499802]]
Accuracy for iterations:  [[0.9747874736785889, 0.8715214729309082, 0.9604726433753967], [0.9732463955879211, 0.8227493762969971, 0.9525769352912903], [0.9721288681030273, 0.818026602268219, 0.9481198787689209]]
F1 for iterations:  [[0.974757759749822, 0.8680192371760987, 0.9604370790274662], [0.9732060844610915, 0.8137439255545337, 0.9525021677645069], [0.9720787442227808, 0.8081970449083345, 0.9480122187614385]]
Precision for iterations:  [[0.9752270522777416, 0.8883265478611913, 0.962133143647721], [0.9739076775914974, 0.8572247322687857, 0.9554433831880542], [0.9730108896022659, 0.855030812765311, 0.9518625471736178]]
Recall for iterations:  [[0.9747874609249467, 0.8715214706096055, 0.9604726265789676], [0.973246370037103, 0.822749400566737, 0.9525769169067232], [0.972128

#### 5B 25%

In [25]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3855986297.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3855986297.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3855986297.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3855986297.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [26]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [27]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Aavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [28]:
global_model = build_model((24,1,1))

In [29]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [30]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [31]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Aavg.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 151ms/step - loss: 0.4503 - accuracy: 0.8421 - val_loss: 0.2403 - val_accuracy: 0.8733
Epoch 2/5
31/31 [==============================] - 4s 143ms/step - loss: 0.1548 - accuracy: 0.9588 - val_loss: 0.1204 - val_accuracy: 0.9766
Epoch 3/5
31/31 [==============================] - 5s 148ms/step - loss: 0.1129 - accuracy: 0.9773 - val_loss: 0.1089 - val_accuracy: 0.9784
Epoch 4/5
31/31 [==============================] - 4s 139ms/step - loss: 0.1032 - accuracy: 0.9792 - val_loss: 0.1059 - val_accuracy: 0.9787
Epoch 5/5
31/31 [==============================] - 4s 139ms/step - loss: 0.0995 - accuracy: 0.9797 - val_loss: 0.0998 - val_accuracy: 0.9788
Epoch 1/5
31/31 [==============================] - 5s 153ms/step - loss: 0.0315 - accuracy: 0.9900 - val_loss: 0.4291 - val_accuracy: 0.6994
Epoch 2/5
31/31 [==============================] - 4s 143ms/step - loss: 0.0222 - accuracy: 0.9910 - val_loss: 0.3429 - val_accuracy: 0.7270
Epoch 3/5
31/

In [32]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.038302790373563766, 0.26699161529541016, 0.06310313194990158], [0.05338504537940025, 0.47174960374832153, 0.0978168398141861], [0.06824193894863129, 0.526358962059021, 0.11863312870264053]]
Accuracy for iterations:  [[0.9821496605873108, 0.866526186466217, 0.9731322526931763], [0.9747143983840942, 0.7648223638534546, 0.9543625116348267], [0.9727351069450378, 0.7611894011497498, 0.9489429593086243]]
F1 for iterations:  [[0.9821416331704106, 0.862501294144424, 0.9731255387637722], [0.9746795893900647, 0.7440076146052567, 0.9542961958400812], [0.9726884720031634, 0.739239173421521, 0.9488402715885467]]
Precision for iterations:  [[0.9822282656806147, 0.8857129164601596, 0.9736063429562448], [0.9752887653650251, 0.8252473062361781, 0.9570148065656969], [0.9735495269555791, 0.8241365933145408, 0.952575202778001]]
Recall for iterations:  [[0.9821496391948348, 0.8665261934171329, 0.9731322670870272], [0.9747144234421105, 0.7648223497783914, 0.9543625190940859], [0.97

#### 5B 5% 

In [33]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3503541895.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3503541895.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_3632/3503541895.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')


In [34]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [35]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Bavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [36]:
global_model = build_model((24,1,1))

In [37]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

KeyboardInterrupt: 

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bavg.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 157ms/step - loss: 0.4760 - accuracy: 0.8449 - val_loss: 0.2409 - val_accuracy: 0.9336
Epoch 2/5
31/31 [==============================] - 5s 156ms/step - loss: 0.1978 - accuracy: 0.9499 - val_loss: 0.1725 - val_accuracy: 0.9616
Epoch 3/5
31/31 [==============================] - 5s 156ms/step - loss: 0.1666 - accuracy: 0.9646 - val_loss: 0.1585 - val_accuracy: 0.9646
Epoch 4/5
31/31 [==============================] - 5s 162ms/step - loss: 0.1535 - accuracy: 0.9671 - val_loss: 0.1502 - val_accuracy: 0.9660
Epoch 5/5
31/31 [==============================] - 5s 156ms/step - loss: 0.1464 - accuracy: 0.9676 - val_loss: 0.1467 - val_accuracy: 0.9658
Epoch 1/5
31/31 [==============================] - 5s 159ms/step - loss: 0.0344 - accuracy: 0.9901 - val_loss: 0.4782 - val_accuracy: 0.6849
Epoch 2/5
31/31 [==============================] - 5s 162ms/step - loss: 0.0227 - accuracy: 0.9915 - val_loss: 0.3628 - val_accuracy: 0.7081
Epoch 3/5
31/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.11124108731746674, 0.2568061649799347], [0.08744512498378754, 0.24105194211006165], [0.07761744409799576, 0.22827744483947754]]
Accuracy for iterations:  [[0.9245961308479309, 0.8307600021362305], [0.9471208453178406, 0.862529993057251], [0.9562432169914246, 0.8848906755447388]]
F1 for iterations:  [[0.9238851317865417, 0.8231112554674732], [0.9468473069984635, 0.858249116698107], [0.9560772557703201, 0.8822572409757126]]
Precision for iterations:  [[0.9330480634765211, 0.8607180094170204], [0.9510947077226397, 0.8822973812218775], [0.9588618004852771, 0.8985446903073606]]
Recall for iterations:  [[0.9245961027199159, 0.8307600087190293], [0.9471208624265973, 0.8625299716631548], [0.9562432440328377, 0.8848906488410957]]


#### 5B 10%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/2097221778.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Bavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Bavg.hdf5')

Epoch 1/5
31/31 [==============================] - 4s 138ms/step - loss: 0.4904 - accuracy: 0.8183 - val_loss: 0.2862 - val_accuracy: 0.9218
Epoch 2/5
31/31 [==============================] - 4s 141ms/step - loss: 0.2706 - accuracy: 0.9273 - val_loss: 0.2487 - val_accuracy: 0.9396
Epoch 3/5
31/31 [==============================] - 4s 138ms/step - loss: 0.2459 - accuracy: 0.9391 - val_loss: 0.2341 - val_accuracy: 0.9417
Epoch 4/5
31/31 [==============================] - 4s 135ms/step - loss: 0.2323 - accuracy: 0.9412 - val_loss: 0.2234 - val_accuracy: 0.9429
Epoch 5/5
31/31 [==============================] - 4s 137ms/step - loss: 0.2236 - accuracy: 0.9424 - val_loss: 0.2185 - val_accuracy: 0.9433
Epoch 1/5
31/31 [==============================] - 4s 141ms/step - loss: 0.0379 - accuracy: 0.9899 - val_loss: 0.3268 - val_accuracy: 0.8140
Epoch 2/5
31/31 [==============================] - 4s 143ms/step - loss: 0.0239 - accuracy: 0.9909 - val_loss: 0.3649 - val_accuracy: 0.7392
Epoch 3/5
31/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.11470375955104828, 0.23222364485263824], [0.09963300824165344, 0.259491503238678], [0.09414619952440262, 0.26222124695777893]]
Accuracy for iterations:  [[0.9249832034111023, 0.8733197450637817], [0.9408031105995178, 0.8527936935424805], [0.9483625292778015, 0.8646552562713623]]
F1 for iterations:  [[0.9242816503937178, 0.8698344379594948], [0.940434626040827, 0.8475370303696735], [0.948103720135536, 0.8604375955006761]]
Precision for iterations:  [[0.9333531466382072, 0.8905279448236121], [0.9458517835799812, 0.8760728745656486], [0.9521770208609084, 0.8846596668569583]]
Recall for iterations:  [[0.9249832013789476, 0.8733197703988956], [0.9408031201612668, 0.8527937222989174], [0.9483624996348126, 0.8646552350504977]]


#### 5B 25%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/3291834707.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Bavg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Bavg.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 145ms/step - loss: 0.5577 - accuracy: 0.7447 - val_loss: 0.4328 - val_accuracy: 0.8525
Epoch 2/5
31/31 [==============================] - 4s 141ms/step - loss: 0.4210 - accuracy: 0.8598 - val_loss: 0.4024 - val_accuracy: 0.8689
Epoch 3/5
31/31 [==============================] - 5s 149ms/step - loss: 0.4061 - accuracy: 0.8665 - val_loss: 0.3930 - val_accuracy: 0.8697
Epoch 4/5
31/31 [==============================] - 6s 192ms/step - loss: 0.3973 - accuracy: 0.8676 - val_loss: 0.3893 - val_accuracy: 0.8705
Epoch 5/5
31/31 [==============================] - 6s 194ms/step - loss: 0.3939 - accuracy: 0.8684 - val_loss: 0.3869 - val_accuracy: 0.8705
Epoch 1/5
31/31 [==============================] - 7s 232ms/step - loss: 0.0492 - accuracy: 0.9897 - val_loss: 0.5851 - val_accuracy: 0.6476
Epoch 2/5
31/31 [==============================] - 11s 345ms/step - loss: 0.0242 - accuracy: 0.9911 - val_loss: 0.4860 - val_accuracy: 0.6481
Epoch 3/5
31

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.13327844440937042, 0.2549484968185425], [0.1026846170425415, 0.2271340936422348], [0.07630041986703873, 0.249038428068161]]
Accuracy for iterations:  [[0.923982560634613, 0.8539562821388245], [0.9342954754829407, 0.8732107877731323], [0.9566011428833008, 0.8710673451423645]]
F1 for iterations:  [[0.9232559208613232, 0.8489066584180305], [0.9338084600112092, 0.8697433202461109], [0.9564431931758209, 0.8673570867825495]]
Precision for iterations:  [[0.932574827909274, 0.876227258613864], [0.9405867705057854, 0.8902485428821898], [0.9590661290639726, 0.8893179846143305]]
Recall for iterations:  [[0.9239825878640918, 0.8539562595364383], [0.9342954804405621, 0.873210782532878], [0.956601127698735, 0.8710673545011989]]


#### 5C NODE 1 5%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/591709827.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C1avg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1avg.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 208ms/step - loss: 0.6010 - accuracy: 0.7845 - val_loss: 0.4593 - val_accuracy: 0.7922
Epoch 2/5
16/16 [==============================] - 3s 197ms/step - loss: 0.3526 - accuracy: 0.8733 - val_loss: 0.2495 - val_accuracy: 0.9253
Epoch 3/5
16/16 [==============================] - 4s 235ms/step - loss: 0.2237 - accuracy: 0.9385 - val_loss: 0.2006 - val_accuracy: 0.9512
Epoch 4/5
16/16 [==============================] - 3s 212ms/step - loss: 0.1972 - accuracy: 0.9572 - val_loss: 0.1820 - val_accuracy: 0.9609
Epoch 5/5
16/16 [==============================] - 3s 200ms/step - loss: 0.1825 - accuracy: 0.9613 - val_loss: 0.1698 - val_accuracy: 0.9639
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0397 - accuracy: 0.9905 - val_loss: 0.6318 - val_accuracy: 0.7277
Epoch 2/5
14/14 [==============================] - 3s 193ms/step - loss: 0.0226 - accuracy: 0.9944 - val_loss: 0.9088 - val_accuracy: 0.7270
Epoch 3/5
14/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06380385905504227, 0.12535728514194489], [0.07164303958415985, 0.15664543211460114], [0.12308940291404724, 0.28510284423828125]]
Accuracy for iterations:  [[0.9883724451065063, 0.9753324389457703], [0.9865245819091797, 0.951100766658783], [0.9668629169464111, 0.895371675491333]]
F1 for iterations:  [[0.9883768370639537, 0.975381199566536], [0.9865325912208416, 0.9512753476524937], [0.9668897656867086, 0.8955566319421113]]
Precision for iterations:  [[0.9884941803977767, 0.9761469231569462], [0.9868968713821396, 0.9559992520165418], [0.9690345154576933, 0.9154998776044775]]
Recall for iterations:  [[0.9883724327324783, 0.9753324129913536], [0.9865245844167226, 0.9511007774467776], [0.9668628940372199, 0.8953716486231199]]


#### 5C NODE 1 10%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_16896/372464333.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C1avg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1avg.hdf5')

Epoch 1/5
16/16 [==============================] - 4s 237ms/step - loss: 0.6154 - accuracy: 0.7681 - val_loss: 0.4970 - val_accuracy: 0.7811
Epoch 2/5
16/16 [==============================] - 3s 216ms/step - loss: 0.4002 - accuracy: 0.8472 - val_loss: 0.3168 - val_accuracy: 0.9028
Epoch 3/5
16/16 [==============================] - 3s 218ms/step - loss: 0.2936 - accuracy: 0.9172 - val_loss: 0.2823 - val_accuracy: 0.9271
Epoch 4/5
16/16 [==============================] - 3s 203ms/step - loss: 0.2698 - accuracy: 0.9332 - val_loss: 0.2664 - val_accuracy: 0.9379
Epoch 5/5
16/16 [==============================] - 4s 220ms/step - loss: 0.2558 - accuracy: 0.9373 - val_loss: 0.2525 - val_accuracy: 0.9390
Epoch 1/5
14/14 [==============================] - 3s 239ms/step - loss: 0.0486 - accuracy: 0.9908 - val_loss: 0.7414 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 3s 206ms/step - loss: 0.0244 - accuracy: 0.9941 - val_loss: 0.8693 - val_accuracy: 0.7270
Epoch 3/5
14/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.06640256196260452, 0.1257031261920929], [0.0543942004442215, 0.10962170362472534], [0.051678989082574844, 0.08576647192239761]]
Accuracy for iterations:  [[0.988379716873169, 0.9754777550697327], [0.9897966384887695, 0.976113498210907], [0.9899938702583313, 0.9845600724220276]]
F1 for iterations:  [[0.9883839596066323, 0.975493552587857], [0.9898015906657185, 0.9761696717141854], [0.9899986226483048, 0.9845855778790374]]
Precision for iterations:  [[0.9884906782301287, 0.9755687720040811], [0.9900111184654423, 0.9773419606883076], [0.9902001605974614, 0.9850820400334382]]
Recall for iterations:  [[0.9883797364807619, 0.9754777301460438], [0.9897966636477841, 0.976113492697813], [0.9899938648514418, 0.9845600523141756]]


#### 5C NODE 1 25%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/2868219575.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C1avg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1avg.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 184ms/step - loss: 0.6350 - accuracy: 0.7102 - val_loss: 0.5472 - val_accuracy: 0.7394
Epoch 2/5
16/16 [==============================] - 3s 207ms/step - loss: 0.4898 - accuracy: 0.8046 - val_loss: 0.4409 - val_accuracy: 0.8484
Epoch 3/5
16/16 [==============================] - 3s 185ms/step - loss: 0.4400 - accuracy: 0.8509 - val_loss: 0.4209 - val_accuracy: 0.8651
Epoch 4/5
16/16 [==============================] - 3s 197ms/step - loss: 0.4258 - accuracy: 0.8624 - val_loss: 0.4109 - val_accuracy: 0.8677
Epoch 5/5
16/16 [==============================] - 3s 199ms/step - loss: 0.4166 - accuracy: 0.8637 - val_loss: 0.4042 - val_accuracy: 0.8683
Epoch 1/5
14/14 [==============================] - 3s 180ms/step - loss: 0.0816 - accuracy: 0.9906 - val_loss: 0.5417 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 177ms/step - loss: 0.0281 - accuracy: 0.9937 - val_loss: 0.7455 - val_accuracy: 0.7270
Epoch 3/5
14/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07301006466150284, 0.13686226308345795], [0.07973063737154007, 0.1810533106327057], [0.11098721623420715, 0.2670384347438812]]
Accuracy for iterations:  [[0.9883286356925964, 0.9682118892669678], [0.9850346446037292, 0.9388033151626587], [0.9744295477867126, 0.9005667567253113]]
F1 for iterations:  [[0.9883327370084766, 0.9681764578223622], [0.9850441636705817, 0.9390300567638474], [0.9744505128900312, 0.9007852290323695]]
Precision for iterations:  [[0.9884330080269008, 0.9682811247686117], [0.9854926668972157, 0.946286857250159], [0.9757355470526499, 0.9189210677814361]]
Recall for iterations:  [[0.9883286102427766, 0.9682118724115382], [0.9850346197668643, 0.938803313231127], [0.9744295772590493, 0.9005667369032915]]


#### 5C NODE 3 5%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/927562370.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C3avg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3avg.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 168ms/step - loss: 0.5297 - accuracy: 0.7412 - val_loss: 1.7559 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 170ms/step - loss: 0.1924 - accuracy: 0.9607 - val_loss: 3.6784 - val_accuracy: 0.1332
Epoch 3/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0802 - accuracy: 0.9893 - val_loss: 3.6751 - val_accuracy: 0.1342
Epoch 4/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0515 - accuracy: 0.9893 - val_loss: 1.8339 - val_accuracy: 0.1357
Epoch 5/5
16/16 [==============================] - 3s 202ms/step - loss: 0.0374 - accuracy: 0.9899 - val_loss: 1.5210 - val_accuracy: 0.1622
Epoch 1/5
14/14 [==============================] - 3s 230ms/step - loss: 0.0231 - accuracy: 0.9949 - val_loss: 0.8074 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 197ms/step - loss: 0.0208 - accuracy: 0.9949 - val_loss: 0.5096 - val_accuracy: 0.7351
Epoch 3/5

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1084146797657013, 0.1123688668012619], [0.1162383109331131, 0.1426282525062561], [0.11805704981088638, 0.16804981231689453]]
Accuracy for iterations:  [[0.9879999160766602, 0.9727712273597717], [0.9890370965003967, 0.952953577041626], [0.9900742173194885, 0.9472498893737793]]
F1 for iterations:  [[0.988003964019539, 0.9727450833085427], [0.9890408833630392, 0.9527551368789061], [0.9900774749113563, 0.9469688561753138]]
Precision for iterations:  [[0.9880929627462135, 0.9728250915394179], [0.9891371143801229, 0.9540567725774562], [0.9901625178749895, 0.9488898335585798]]
Recall for iterations:  [[0.9879999415700137, 0.9727711981399404], [0.9890370738262877, 0.9529535711690765], [0.9900742060825616, 0.9472498728474896]]


#### 5C NODE 3 10%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/4114672810.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C3avg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3avg.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 212ms/step - loss: 0.5324 - accuracy: 0.7423 - val_loss: 1.7323 - val_accuracy: 0.0233
Epoch 2/5
16/16 [==============================] - 3s 182ms/step - loss: 0.1918 - accuracy: 0.9744 - val_loss: 3.9565 - val_accuracy: 0.1281
Epoch 3/5
16/16 [==============================] - 3s 176ms/step - loss: 0.0867 - accuracy: 0.9892 - val_loss: 3.8688 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0524 - accuracy: 0.9892 - val_loss: 1.6523 - val_accuracy: 0.1386
Epoch 5/5
16/16 [==============================] - 3s 163ms/step - loss: 0.0364 - accuracy: 0.9891 - val_loss: 1.2198 - val_accuracy: 0.2383
Epoch 1/5
14/14 [==============================] - 3s 184ms/step - loss: 0.0220 - accuracy: 0.9948 - val_loss: 0.8367 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 171ms/step - loss: 0.0195 - accuracy: 0.9950 - val_loss: 0.6023 - val_accuracy: 0.7283
Epoch 3/5
14/

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07911981642246246, 0.10972201079130173], [0.11108022183179855, 0.12621992826461792], [0.10594591498374939, 0.14654435217380524]]
Accuracy for iterations:  [[0.9884600639343262, 0.9746421575546265], [0.989482581615448, 0.9660139679908752], [0.9902129769325256, 0.9479401111602783]]
F1 for iterations:  [[0.9884641874354215, 0.9746253145543056], [0.9894866514296236, 0.9659476699492876], [0.9902162674857102, 0.9476694014296102]]
Precision for iterations:  [[0.9885660727407686, 0.9746612204195951], [0.9896081802206068, 0.9662745375191877], [0.9903052955349512, 0.9495162369660378]]
Recall for iterations:  [[0.9884600777118817, 0.9746421565065756], [0.9894826024715884, 0.9660139504468502], [0.9902129772999503, 0.9479401293322677]]


#### 5C NODE 3 25%

In [ ]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_21336/512859027.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')


In [ ]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [ ]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C3avg.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [ ]:
global_model = build_model((24,1,1))

In [ ]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [ ]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)
    loss_complete, accuracy_complete, f1_complete, precision_complete, recall_complete = evaluation(global_model, xcomplete, ycomplete)

    loss_it.append([loss_basic, loss_plus, loss_complete])
    accuracy_it.append([accuracy_basic, accuracy_plus, accuracy_complete])
    f1_it.append([f1_basic, f1_plus, f1_complete])
    precision_it.append([precision_basic, precision_plus, precision_complete])
    recall_it.append([recall_basic, recall_plus, recall_complete])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3avg.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 181ms/step - loss: 0.5393 - accuracy: 0.7489 - val_loss: 1.6758 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 186ms/step - loss: 0.2027 - accuracy: 0.9544 - val_loss: 3.6085 - val_accuracy: 0.1320
Epoch 3/5
16/16 [==============================] - 3s 173ms/step - loss: 0.0847 - accuracy: 0.9892 - val_loss: 4.0488 - val_accuracy: 0.1337
Epoch 4/5
16/16 [==============================] - 3s 204ms/step - loss: 0.0519 - accuracy: 0.9894 - val_loss: 2.0923 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 206ms/step - loss: 0.0370 - accuracy: 0.9903 - val_loss: 1.6371 - val_accuracy: 0.1401
Epoch 1/5
14/14 [==============================] - 3s 212ms/step - loss: 0.0219 - accuracy: 0.9950 - val_loss: 0.8711 - val_accuracy: 0.7271
Epoch 2/5
14/14 [==============================] - 3s 219ms/step - loss: 0.0203 - accuracy: 0.9950 - val_loss: 0.6223 - val_accuracy: 0.7294
Epoch 3/5

In [ ]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08544706553220749, 0.12328588217496872], [0.09680859744548798, 0.13693860173225403], [0.12291232496500015, 0.15408408641815186]]
Accuracy for iterations:  [[0.9878538846969604, 0.9722262620925903], [0.9889128804206848, 0.9629804491996765], [0.9890589714050293, 0.9475768208503723]]
F1 for iterations:  [[0.9878578666501733, 0.9722010170737153], [0.988916709205785, 0.9628907034001225], [0.9890627905095888, 0.9472986746263102]]
Precision for iterations:  [[0.9879434565329552, 0.9722715522619876], [0.9890099785924518, 0.9633804123844554], [0.9891592171717783, 0.949208839513966]]
Recall for iterations:  [[0.9878538666043414, 0.9722262588098525], [0.9889129101054661, 0.9629804548426942], [0.9890589850711385, 0.9475768364455424]]
